## 案例分析一：Kaggle Competition | Titanic Machine Learning from Disaster


***本作基于： [Titanic](https://github.com/agconti/kaggle-titanic). 原数据来源： [Kaggle.com](http://www.kaggle.com/c/titanic-gettingStarted).***

In [ ]:
import numpy as np
import pandas as pd

#### `import … as …`是给我们引入的包加上缩写，在后续调用的时候可以直接用缩写调用

In [ ]:
df = pd.read_csv("train.csv") 

In [ ]:
df #直接打变量名字就可以预览数据内容

In [ ]:
df = df.drop(['Ticket','Cabin'], axis=1)
df = df.dropna() 

#### `.drop()`是用于删除整一列数据，不作额外判断。  
* 如果将`axis`参数改为1 (默认为0)，则会沿纵向删除数据  
#### `.dropna()`用于删除空值NaN  
* 在不额外调整`axis`参数值时，同样默认遍历全部行进行删除

## 案例分析二： Data Cleaning with NumPy and Pandas
  
***本作基于： [Data-cleaning](https://github.com/mramshaw/Data-Cleaning). 原数据来源： [Pandas and NumPy](https://realpython.com/python-data-cleaning-numpy-pandas/).***

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('BL-Flickr-Images-Book.csv')
df.head()

#### `df.head()`意为预览前五列数据
* 括号中定义数字可以预览更多行的数据

In [ ]:
to_drop = ['Edition Statement',
          'Corporate Author',
          'Corporate Contributors',
          'Former owner',
          'Engraver',
          'Contributors',
          'Issuance type',
          'Shelfmarks']
df.drop(to_drop, inplace=True, axis=1)
df.head()

#### `df.drop(labels= ，inplace=false，axis=0，errors='raise')`是drop函数的部分语法默认值
* labels：在这里用`to_drop`统一标注想要删除的列；在较小的数据集中，可以直接用字符串标明需要删除的列名
* axis：默认值为0，按纵向删除；在此改为1，按列方向删除
* inplace：默认值为`false`；如果将参数改为`True`，将会直接覆盖原数据，操作不可逆
* errors：默认值为`'raise'`，此时如果出现不存在的列名则返回`KeyError`；参数改为'ignore'便可以忽略打错的列名

In [ ]:
df['Identifier'].is_unique

#### `df['Identifier'].is_unique` 是测试某列名是否是唯一的，因为我们想要把在后续以这个列名为基础查看数据

In [ ]:
df = df.set_index('Identifier')
# df.set_index('Identifier', inplace=True) 效果一样
df.head()

#### `df.set_index('Identifier')` 是将检索值固定为了我们想要的，替换掉了默认的0,1,2…

In [ ]:
df.loc[216]
# df.iloc[1]

#### `df.loc[]` 是查询特定的检索值所在行的数据，在这里是Identifier的编号
* 如果loc[]查找的数值不在范围内，会返回'KeyError'
#### `df.iloc[]` 是查询相对的检索值所在的数据，默认值是`0`，即第一行的数据

In [ ]:
# df.get_dtype_counts()--》老版
df.dtypes.value_counts()

#### `df.dtypes.value_counts()` 是查询数据中存在的数据类型
* object：是主要关注的结果
* Name：在这句代码中基本上可以忽略
* dtype：指的是这句代码统计结果的储存方式是整数->在计算比例的时候可能统计结果会出现float64的情况

In [ ]:
df.loc[1905:, 'Date of Publication'].head(10)

通过这样检索“发行日期”的数据，发现格式不是统一的

In [ ]:
regex = r'^(\d{4})'
extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)
extr.head(10)

#### `r'^(\d{4})'` 是严格匹配字符串前四位数字的正则表达式
* `r' '`表示原始字符串，避免`\d`被python误识别
* `^`:是匹配字符串的起始符
* `\d`：意为匹配任意数字（依赖后续条件）
* `{n}`：意为匹配前一个字符n次
* `（）`：将匹配的内容捕捉为一个分组，用于后续提取  
由于正则表达式，笔者仍掌握不太熟练，其他语法元素可以参考: ***[CDSN论坛](https://blog.csdn.net/weixin_42448623/article/details/102785880)***
#### `.str.extract(r'^(\d{4})', expand=False)`用于从正则表达式中提取匹配的内容
* 在正则表达式位置，可以用提前定义好的变量名，在这里可以用`rf^{regex}`来替换原本表达式的参数位置
* expand：默认值为`True`，在匹配完数据后返回原数据+匹配后的数据；改为`False`后只返回匹配后的数据，匹配失败的返回`NaN`

In [ ]:
df['Date of Publication'] = pd.to_numeric(extr,errors='coerce')
df['Date of Publication'].dtype

#### `pd.to_numeric()` 是将目标数据强制转换为数值类型（比如原来是字符串）
* `error`：设定参数为`coerce`可以将无效值转换为`NaN`；`ignore`可以保留原始数据类型，但是不建议
* 如果全部值都可以转换为整数，则`.dtype`结果会返回`int64`；如果存在NaN，那么会转换为`float64`

In [ ]:
df['Date of Publication'].head()

至此，我们格式化了出版日期的年份。接下来开始格式化出版地名的数据

In [ ]:
df['Place of Publication'].head(10)

In [ ]:
pub = df['Place of Publication']
london = pub.str.contains('London')
london[:10]

#### `str.contains()` 是用于检查字符串列是否包含特定子串（可以是正则表达式）
* pat：要匹配的是字符串还是正则表达式
* case：默认为`True`，意为是否区分大小写
* flags：正则表达式的标志（如忽略大小写）
* na：处理缺失值的方式，默认将`NaN`处理为`False`
* regex：默认为`True`，即视为正则表达式；在这里因为无需匹配额外的符号等，所以用True和False没有区别
  
在这里结果是：精确匹配是否存在字符串为`"London"`  
如果改为`.str.contains('London', case=False)`，则可以查到登记为`"london"`的出版地


In [ ]:
oxford = pub.str.contains('Oxford')
df['Place of Publication'] = np.where(london, 'London',
                                      np.where(oxford, 'Oxford',
                                      pub.str.replace('-', ' ')))
df['Place of Publication'].head(20)

#### `np.where()` 是为了满足：根据不同条件返回不同的值
* 参数填充顺序为：  
      i.条件1，满足条件时的填充值；  
      ii.用嵌套自身的方式填充：条件2，满足条件2时填充的值  
      iii.在嵌套的最末尾填上如果不满足时，需要填充的值
  
* 此处的判断逻辑为：
      i.检查在检索的列中，是否存在`London`，满足的话填充"London"  
      ii.如果没有`London`，转入第二层，检查是否存在`Oxford`，满足的话填充"Oxford"  
      iii.如果都不满足，便仅将"-"替换为"空格"  


至此，我们就完成了对出版日期和出版地的数据清洗。

## 案例分析三： NYC taxi Data Cleaning  

***原数据来源： [NYC Taxi](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).***

In [2]:
# pip install pyarrow
import pandas as pd
import numpy as np
df = pd.read_csv("C:\\Users\\Kris\\Github类_自用\\dataset\\Case 3\\yellow_tripdata_2014-01.csv")

In [ ]:
print(df.columns)

In [3]:
dfs = df.shape[0]
print("目前有:", dfs,'列数据')

df = df[(df[' passenger_count'].between(0,4))]
# df = [(df[' passenger_count'] >= 0) & (df[' passenger_count'] <= 4)]

print("目前有:", df.shape[0],'列数据',"清洗了",dfs-df.shape[0],"列数据")

目前有: 11830324 列数据
目前有: 10501946 列数据 清洗了 1328378 列数据


#### `df = df[(df[' '].between())]` 是用于筛出特定标签的特定数据范围-在这里用于筛除异常值
* 用`.between`会比用大于等于或者小于等于更简洁
* 记住不能使用方括号，使用`.between`方法已经是闭区间
* 注：纽约的出租车不一定只能载四个人，在这里仅作练习目的

In [4]:
df = df.drop(['vendor_id',' pickup_datetime', ' dropoff_datetime',
              ' passenger_count',' rate_code', ' store_and_fwd_flag',
              ' payment_type',' fare_amount', ' surcharge',
              ' mta_tax',' tip_amount',' tolls_amount',' total_amount'], axis=1)

#### `df.shape[0]` 是用于筛出特定标签的特定数据范围-在这里用于筛除异常值
* 作为属性，而非方法，不用括号调用
* 如果不加`[0]`，能够返回行和列的信息，这里仅需要了解删除了多少数据
* 等价于`len(df)`
* 区别于：`df.size`，这个属性会输出全部的元素数量

In [5]:
dfs = df.shape[0]
print("目前有:", dfs,'列数据')

df = df[~((df[' pickup_latitude'] == 0) &
          (df[' pickup_longitude'] == 0) &
          (df[' dropoff_latitude'] == 0) &
          (df[' dropoff_longitude'] == 0))]



目前有: 10501946 列数据


#### `df = df[~(df['column'] == 0)]` 将满足特定条件标记为Ture，用于后续的的取反筛除
* 使用了按位与运算符将同时满足四个条件的行标记为 True，再取反（~）得到要保留的行
* &：按位与，用于同时满足多个条件
* |：按位或，用于满足任一条件
* ^：按位异或，两个布尔值不同返回 True，相同返回 False

In [6]:
dfs = df.shape[0]
print("目前有:", dfs,'列数据')

df = df.dropna()

print("目前有:", df.shape[0],'列数据',"清洗了",dfs-df.shape[0],"列数据")

目前有: 10367995 列数据
目前有: 10367921 列数据 清洗了 74 列数据


In [7]:
dfs = df.shape[0]
print("目前有:", dfs,'列数据')

import math

# Haversine 计算地理直线距离（单位：公里）
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 地球半径（km）
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

# 1. 定义完整数据：pickup 和 dropoff 坐标都不为 0
df_full = df[(df[' pickup_latitude'] != 0) & (df[' pickup_longitude'] != 0) &
             (df[' dropoff_latitude'] != 0) & (df[' dropoff_longitude'] != 0)].copy()

# 计算完整数据中两端的直线距离
df_full.loc[:, 'geo_dist'] = haversine(
    df_full[' pickup_latitude'], df_full[' pickup_longitude'],
    df_full[' dropoff_latitude'], df_full[' dropoff_longitude']
)

# 计算行驶距离与直线距离的比值，并取中位数作为代表
df_full.loc[:, 'ratio'] = df_full[' trip_distance'] / (df_full['geo_dist'] + 1e-5)
ratio_mean = df_full['ratio'].median()

# 2. 定义缺失数据：
#    缺失 pickup：pickup 坐标为 0，但 dropoff 坐标存在
df_missing_pickup = df[(df[' pickup_latitude'] == 0) & (df[' pickup_longitude'] == 0) & 
                       (df[' dropoff_latitude'] != 0) & (df[' dropoff_longitude'] != 0)].copy()

#    缺失 dropoff：dropoff 坐标为 0，但 pickup 坐标存在
df_missing_dropoff = df[(df[' dropoff_latitude'] == 0) & (df[' dropoff_longitude'] == 0) & 
                        (df[' pickup_latitude'] != 0) & (df[' pickup_longitude'] != 0)].copy()

# 定义在圆上随机生成点的函数：以 (lat, lon) 为圆心，distance_km 为半径随机生成一个点
def simulate_point(lat, lon, distance_km):
    angle = np.random.uniform(0, 2 * np.pi)
    # 简单换算：1° 约等于 111 km
    dx = (distance_km / 111) * np.cos(angle)
    dy = (distance_km / 111) * np.sin(angle)
    return lat + dy, lon + dx

# 3. 补全缺失的 pickup 坐标
pickup_lat_sim = []
pickup_lon_sim = []
for i, row in df_missing_pickup.iterrows():
    # 根据 trip_distance 和 ratio_mean 估计直线距离（单位 km）
    est_geo_dist = row[' trip_distance'] / ratio_mean
    # 以 dropoff 坐标为圆心生成随机点
    lat, lon = simulate_point(row[' dropoff_latitude'], row[' dropoff_longitude'], est_geo_dist)
    pickup_lat_sim.append(lat)
    pickup_lon_sim.append(lon)

df_missing_pickup.loc[:, ' pickup_latitude'] = pickup_lat_sim
df_missing_pickup.loc[:, ' pickup_longitude'] = pickup_lon_sim

# 4. 补全缺失的 dropoff 坐标
dropoff_lat_sim = []
dropoff_lon_sim = []
for i, row in df_missing_dropoff.iterrows():
    est_geo_dist = row[' trip_distance'] / ratio_mean
    # 以 pickup 坐标为圆心生成随机点
    lat, lon = simulate_point(row[' pickup_latitude'], row[' pickup_longitude'], est_geo_dist)
    dropoff_lat_sim.append(lat)
    dropoff_lon_sim.append(lon)

df_missing_dropoff.loc[:, ' dropoff_latitude'] = dropoff_lat_sim
df_missing_dropoff.loc[:, ' dropoff_longitude'] = dropoff_lon_sim

# 5. 合并所有数据：完整数据、补全 pickup 和补全 dropoff 的数据
df = pd.concat([df_full, df_missing_pickup, df_missing_dropoff], ignore_index=True)

print("目前有:", df.shape[0],'列数据',"清洗了",dfs-df.shape[0],"列数据")

目前有: 10367921 列数据
目前有: 10367913 列数据 清洗了 8 列数据


In [8]:
dfs = df.shape[0]
print("目前有:", dfs,'列数据')

df = df[(df[' pickup_latitude'].between(40.4774, 40.9155) & 
        df[' pickup_longitude'].between(-74.2597, -73.700) &
        df[' dropoff_latitude'].between(40.4774, 40.9155) &
        df[' dropoff_longitude'].between(-74.2597, -73.700))]

print("目前有:", df.shape[0],'列数据',"清洗了",dfs-df.shape[0],"列数据")

目前有: 10367913 列数据
目前有: 10360154 列数据 清洗了 7759 列数据


In [9]:
df = df.drop(['geo_dist', 'ratio'], axis=1)

In [14]:
#pip install folium
import folium
from folium.plugins import FastMarkerCluster, HeatMap
from selenium import webdriver
import time

df_sample = df.head(10000)

# 以纽约市中心为地图初始中心
ny_center = [40.7128, -74.0060]
m = folium.Map(location=ny_center, zoom_start=12)

# 构造两个列表，分别存放 pickup 和 dropoff 的经纬度数据
pickup_locations = df_sample[[' pickup_latitude', ' pickup_longitude']].values.tolist()
dropoff_locations = df_sample[[' dropoff_latitude', ' dropoff_longitude']].values.tolist()

# -------------------------------
# 1. 使用 FastMarkerCluster 聚合大量数据点
# -------------------------------
FastMarkerCluster(data=pickup_locations, name='Pickup FastCluster').add_to(m)
FastMarkerCluster(data=dropoff_locations, name='Dropoff FastCluster').add_to(m)

# -------------------------------
# 2. 添加 HeatMap 热力图展示数据密度
# -------------------------------
# 这里使用 radius 和 blur 参数调整热力图的视觉效果
HeatMap(data=pickup_locations, name='Pickup HeatMap', radius=8, blur=10, min_opacity=0.5).add_to(m)
HeatMap(data=dropoff_locations, name='Dropoff HeatMap', radius=8, blur=10, min_opacity=0.5).add_to(m)

# 添加图层控制（可选）
folium.LayerControl().add_to(m)

# 保存地图为 HTML 文件，方便在浏览器中查看
m.save("ny_taxi_distribution.html")